In [12]:
! pip install transformers peft accelerate datasets bitsandbytes


   ---------------------------------------- 0.0/72.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.2 MB ? eta -:--:--
   ---------------------------------------- 0.8/72.2 MB 1.7 MB/s eta 0:00:43
    --------------------------------------- 1.0/72.2 MB 1.8 MB/s eta 0:00:40
    --------------------------------------- 1.6/72.2 MB 2.0 MB/s eta 0:00:37
   - -------------------------------------- 2.1/72.2 MB 2.0 MB/s eta 0:00:36
   - -------------------------------------- 2.4/72.2 MB 1.9 MB/s eta 0:00:36
   - -------------------------------------- 2.9/72.2 MB 2.0 MB/s eta 0:00:36
   - -------------------------------------- 3.4/72.2 MB 2.0 MB/s eta 0:00:35
   -- ------------------------------------- 3.9/72.2 MB 2.1 MB/s eta 0:00:33
   -- ------------------------------------- 4.5/72.2 MB 2.1 MB/s eta 0:00:32
   -- ------------------------------------- 5.0/72.2 MB 2.2 MB/s eta 0:00:31
   --- ------------------------------------ 5.5/72.2 MB 2.3 MB/s eta 0:00:30
   --- ------

In [6]:
from transformers import (
    AutoTokenizer, AutoModelForQuestionAnswering,
    TrainingArguments, Trainer, default_data_collator
)
from datasets import load_dataset
import logging

logging.basicConfig(level=logging.INFO)

# 1. Load dataset từ file SQuAD json
data_files = {
    "train": "../train.json",
    "validation": "../valid.json"
}
dataset = load_dataset("json", data_files=data_files)

# 2. Load tokenizer và model
model_ckpt = "deepset/xlm-roberta-base-squad2"  # tốt cho tiếng Việt
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

# 3. Tokenize function
def preprocess_function(examples):
    return tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# 4. TrainingArguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)


# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

# 6. Train
trainer.train()


C:\Users\Windows\AppData\Local\Temp\ipykernel_15132\787550989.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: The model did not return a loss from the inputs, only the following keys: start_logits,end_logits. For reference, the inputs it received are input_ids,attention_mask.

In [7]:
print(dataset["train"].column_names)


['id', 'context', 'question', 'answers']


In [5]:
import transformers
print(transformers.__version__)


4.53.0


In [24]:
pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("goldfish-models/vie_latn_10mb")
model = AutoModelForQuestionAnswering.from_pretrained(
    "goldfish-models/vie_latn_10mb",
    device_map="auto",
    load_in_8bit=True,
    # Cần phải cung cấp tokenizer.pad_token_id cho AutoModelForQuestionAnswering
    # nếu nó không tự xác định được.
    pad_token_id=tokenizer.pad_token_id
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.
Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at goldfish-models/vie_latn_10mb and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
! pip install peft 

In [20]:
# set up lora 
from peft import LoraConfig, get_peft_model, TaskType

pert_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r = 8, 
    lora_alpha = 16, 
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, pert_config)
model

c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\tuners\tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): GPT2LMHeadModel(
          (transformer): GPT2Model(
            (wte): Embedding(38912, 512)
            (wpe): Embedding(512, 512)
            (drop): Dropout(p=0.1, inplace=False)
            (h): ModuleList(
              (0-3): 4 x GPT2Block(
                (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (attn): GPT2Attention(
                  (c_attn): lora.Linear(
                    (base_layer): Conv1D(nf=1536, nx=512)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=1536, bias=F

In [ ]:
from datasets import load_dataset 

data = load_dataset()

{'input_ids': tensor([[38516,     9,  2762,    21, 38517]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [7]:
output = model.generate(
    input_ids = token["input_ids"], 
    max_length = 100,
    do_sample = True,
    top_k = 50, 
    top_p = 0.95,
    temperature = 0.8,
    repetition_penalty=1.2,
    num_return_sequences=1
    )

output[0]

tensor([38516,     9,  2762,    21, 38517,   288,  1715,   841,   176,   691,
          122,     8,    85,   214,   704,   425,    25,    11,    18,   291,
          209,    85,   214,    43,    46,     5,   549,   384,    10,   247,
          160,    33,   704,   425,   309,   587,     4, 38517])

In [9]:
tokenizer.decode(output[0])

'[CLS] Chào bạn[SEP] Chắc mắc bệnh viện cấp là điều trị bác sĩ sẽ được những phương pháp điều trị tại nhà và hỗ trợ các đơn vị về bác sĩ chuyên khoa.[SEP]'

In [7]:
prompt = "Tôi bị đau đầu và mệt mỏi, có phải là triệu chứng của"

inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(
    input_ids=inputs["input_ids"],
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2,
    num_return_sequences=1
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Tôi bị đau đầu và mệt mỏi, có phải là triệu chứng của Để điều trị bệnh nhân có thể được hiệu quả về tình trạng viêm da. Theo thông tin, việc sử dụng thuốc trong bài viết này để làm tăng sức khỏe cho trẻ em nên cần tìm thấy hiệu quả tốt nhất để bạn và các trường hợp của mình và cách chữa viêm xoang. Nếu bạn sẽ khiến nó với một số loại bỏ qua những bệnh tim ở những người bệnh tật để đạt hơn rất nhiều loại bỏ


In [ ]:
# Load model directly


context="""
Trường Đại học Công nghệ Thông tin (ĐH CNTT), Đại học Quốc gia Thành phố Hồ Chí Minh (ĐHQG-HCM) là trường đại học công lập đào tạo về công nghệ thông tin và truyền thông (CNTT&TT) được thành lập theo quyết định số 134/2006/QĐ-TTg ngày 08/06/2006 của Thủ tướng Chính phủ. Là trường thành viên của ĐHQG-HCM, trường ĐH CNTT có nhiệm vụ đào tạo nguồn nhân lực công nghệ thông tin chất lượng cao, góp phần tích cực vào sự phát triển của nền công nghiệp công nghệ thông tin Việt Nam, đồng thời tiến hành nghiên cứu khoa học và chuyển giao công nghệ thông tin tiên tiến, đặc biệt là hướng vào các ứng dụng nhằm góp phần đẩy mạnh sự nghiệp công nghiệp hóa, hiện đại hóa đất nước.
Sau hơn 10 năm xây dựng và phát triển, hiện trường ĐH CNTT sở hữu cơ sở vật chất gồm khu học tập, nghiên cứu và làm việc được đầu tư xây dựng khang trang, hiện đại với tổng diện tích trên 14 hecta trong khuôn viên khu đô thị ĐHQG-HCM.
"""
question="""
Trường UIT mang trong mình nhiệm vụ gì?
"""

tokenizer = AutoTokenizer.from_pretrained("PhucDanh/vit5-fine-tuning-for-question-answering")
try:
  tokenizer.model_input_names.remove("token_type_ids")
except:
  print("already removed!!!")

inputs = tokenizer(question, context, return_tensors="pt")


with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print(tokenizer.decode(predict_answer_tokens))


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


đào tạo nguồn nhân lực công nghệ thông tin chất lượng cao, góp phần tích cực vào sự phát triển của nền công nghiệp công nghệ thông tin Việt Nam, đồng thời tiến hành nghiên cứu khoa học và chuyển giao công nghệ thông tin tiên tiến, đặc biệt là hướng vào các ứng dụng nhằm góp phần đẩy mạnh sự nghiệp công nghiệp hóa, hiện đại hóa đất nước


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("PhucDanh/vit5-fine-tuning-for-question-answering")
import torch
dataset = load_dataset("parquet", data_files={"train": "../Data/Dataset/train-00000-of-00001.parquet", 
                                               "valid": "../Data/Dataset/validation-00000-of-00001.parquet",
                                               "test": "../Data/Dataset/test-00000-of-00001.parquet"})

def preprocess(example):
    prompt = f"Answer the question: {example['question']} Context: {example['context']}"
    target = example["answer"]
    return {"input_text": prompt, "target_text": target}

dataset = dataset.map(preprocess)

tokenizer = AutoTokenizer.from_pretrained("PhucDanh/vit5-fine-tuning-for-question-answering")

def tokenize(example):
    model_inputs = tokenizer(
        example["input_text"], padding="max_length", truncation=True, max_length=512
    )
    labels = tokenizer(
        example["target_text"], padding="max_length", truncation=True, max_length=64
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/39881 [00:00<?, ? examples/s]

Map:   0%|          | 0/2215 [00:00<?, ? examples/s]

Map:   0%|          | 0/2217 [00:00<?, ? examples/s]

In [6]:
pip install transformers==4.40.1

   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.0 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.0 MB 3.5 MB/s eta 0:00:03
   --------- ------------------------------ 2.1/9.0 MB 3.6 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.0 MB 3.7 MB/s eta 0:00:02
   ----------------- ---------------------- 3.9/9.0 MB 3.8 MB/s eta 0:00:02
   -------------------- ------------------- 4.7/9.0 MB 3.9 MB/s eta 0:00:02
   ------------------------- -------------- 5.8/9.0 MB 3.9 MB/s eta 0:00:01
   ----------------------------- ---------- 6.6/9.0 MB 4.0 MB/s eta 0:00:01
   -------------------------------- ------- 7.3/9.0 MB 3.9 MB/s eta 0:00:01
   ------------------------------------ --- 8.1/9.0 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------  8.9/9.0 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 3.8 MB/s eta 0:00:00
   ----------------

  You can safely remove it manually.


In [1]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./vit5-qa-checkpoint",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=2000,
    eval_steps=2000,
    save_total_limit=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=4,
    logging_dir="./logs",
    predict_with_generate=True,
    load_best_model_at_end=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

RuntimeError: Failed to import transformers.trainer_seq2seq because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\__init__.py)